In [ ]:
# from __future__ import print_function
import numpy as np
import csv
import time
import datetime

class Board(object):
    """
    board for the game
    """
    
    def __init__(self, **kwargs):
        #self.coinNameArray = np.array()
        self.coinNameArray = ["bitcoin", "ethereum", "litecoin", "neo", "monero", "ripple", "dash"]
        self.width = int(kwargs.get('width', 288))
        self.height = int(kwargs.get('height', len(self.coinNameArray)))
        self.states = {} # board states, key:move as location on the board, value:player as pieces type
        self.state3d = np.zeros((self.width*self.height))
        self.n_in_row = int(kwargs.get('n_in_row', 5)) # need how many pieces in a row to win
        self.players = [1, 2] # player1 and player2
        self.points1 = 0
        self.points2 = 0
        self.column = 0
        self.timeStampArray = []
        self.usdPriceArray = []
        self.marketCapArray = []
        self.volumeArray = []
        self.hourChangeArray = []
        self.dayChangeArray = []
        self.weekChangeArray = []
        self.nextChangeArray = []
        
        for name in self.coinNameArray:
            with open(name.lower() + 'PriceData.csv','r') as csvfile:
                priceReader = csv.reader(csvfile, delimiter=',')
                i = 0
                for row in priceReader:
                    if i == 0:
                        self.timeStampArray.append(row)
                    elif i == 1:
                        self.usdPriceArray.append(row)
                    elif i == 2:
                        self.marketCapArray.append(row)
                    elif i == 3:
                        self.volumeArray.append(row)
                    elif i == 4:
                        self.hourChangeArray.append(row)
                    elif i == 5:
                        self.dayChangeArray.append(row)
                    elif i == 6:
                        self.weekChangeArray.append(row)
                    elif i == 7:
                        self.nextChangeArray.append(row)
                    i = i + 1
            #print(name)
        #print("__init__")
        
    def init_board(self, start_player=0):
        if self.width < self.n_in_row or self.height < self.n_in_row:
            raise Exception('board width and height can not less than %d' % self.n_in_row)
        self.current_player = self.players[start_player]  # start player        
        self.availables = list(range(self.width * self.height)) # available moves 
        self.states = {} # board states, key:move as location on the board, value:player as pieces type
        self.last_move = -1
        #print("init_board")

    def move_to_location(self, move):
        """       
        3*3 board's moves like:
        6 7 8
        3 4 5
        0 1 2
        and move 5's location is (1,2)
        """
        h = move  // self.width
        w = move  %  self.width
        #print("move_to_location")
        return [h, w]

    def location_to_move(self, location):
        if(len(location) != 2):
            return -1
        h = location[0]
        w = location[1]
        move = h * self.width + w
        #print("location_to_move")
        if(move not in range(self.width * self.height)):
            return -1
        return move

    def current_state(self): 
        """return the board state from the perspective of the current player
        shape: 4*width*height"""
        
        square_state = np.zeros((11, self.width, self.height))
        self.state3d = np.zeros((self.width*self.height))
        if self.states:
            moves, players = np.array(list(zip(*self.states.items())))
            move_curr = moves[players == self.current_player]
            move_oppo = moves[players != self.current_player]                           
            square_state[0][move_curr // self.width, move_curr % self.height] = 1.0
            square_state[1][move_oppo // self.width, move_oppo % self.height] = 1.0   
            square_state[2][self.last_move //self.width, self.last_move % self.height] = 1.0 # last move indication   
        if len(self.states)%2 == 0:
            square_state[3][:,:] = 1.0
         
        x = 0
        while(x < self.width):
            count = 0
            if x < self.width:
                for name in self.coinNameArray:
                    #square_state[4][count][x] = self.coinNameArray[count];
                    #square_state[4,count,x] = self.timeStampArray[count][x+1];
                    square_state[4,x,count] = self.usdPriceArray[count][x+1]
                    #print(4)
                    square_state[5,x,count] = self.marketCapArray[count][x+1]
                    #print(5)
                    square_state[6,x,count] = self.volumeArray[count][x+1]
                    #print(6)
                    square_state[7,x,count] = self.hourChangeArray[count][x+1]
                    #print(7)
                    square_state[8,x,count] = self.dayChangeArray[count][x+1]
                    #print(8)
                    square_state[9,x,count] = self.weekChangeArray[count][x+1]
                    #print(9)
                    #print(float(self.nextChangeArray[count][x+1]))
                    #square_state[10,x,count] = self.nextChangeArray[count][x+1]
                    np.insert(square_state, [10,x,count], self.nextChangeArray[count][x+1])
                    #print(square_state[10,x,count])
                    self.state3d[count * self.width + x] = square_state[10,x,count]
                    #print(self.state3d[x,count])
                    #print(10)
                    count = count + 1
                    #print("state")
                    #print(name)
                x = x + 1
        #print("current_state")
        return square_state[:,::-1,:]

    def do_move(self, move):
        self.states[move] = self.current_player
        self.availables.remove(move)
        if self.current_player == self.players[1]:
            #print("before")
            print(self.state3d[move])
            print(self.state3d[self.last_move])
            if float(self.state3d[move]) > float(self.state3d[self.last_move]) :
                self.points2 = self.points2 + 1
            #print("middle")
            if float(self.state3d[move]) < float(self.state3d[self.last_move]):
                self.points1 = self.points1 + 1
            #print("end")
            for i in range(0, self.height):
                tempMove = i * self.width + self.column
                if tempMove in self.availables:
                    self.availables.remove(tempMove)
            self.column = self.column + 1
        self.current_player = self.players[0] if self.current_player == self.players[1] else self.players[1] 
        self.last_move = move
        #print("do_move")
        
    def has_a_winner(self):
        width = self.width
        height = self.height
        states = self.states
        n = self.n_in_row
        #print("has_a_winner")
        if self.column == width-1:
            if self.points1 > self.points2:
                player = self.players[0]
            elif self.points1 < self.points2:
                player = self.players[1]
            else:
                player = 0
            print("win")
            return True, player
        print(str(self.points1))
        print(str(self.points2))
        print(str(self.column))
        return False, -1

    def game_end(self):
        """Check whether the game is ended or not"""
        win, winner = self.has_a_winner()
        #print("game_end")
        if win:
            return True, winner
        elif not len(self.availables):#            
            return True, -1
        return False, -1

    def get_current_player(self):
        return self.current_player


class Game(object):
    """
    game server
    """

    def __init__(self, board, **kwargs):
        self.board = board
        #print("__init__")

    def graphic(self, board, player1, player2):
        """
        Draw the board and show game info
        """
        width = board.width
        height = board.height

        print("Player", player1, "with X".rjust(3))
        print("Player", player2, "with O".rjust(3))
        print()
        for x in range(width):
            print("{0:8}".format(x), end='')
        print('\r\n')
        for i in range(height - 1, -1, -1):
            print("{0:4d}".format(i), end='')
            for j in range(width):
                loc = i * width + j
                p = board.states.get(loc, -1)
                if p == player1:
                    print('X'.center(8), end='')
                elif p == player2:
                    print('O'.center(8), end='')
                else:
                    print('_'.center(8), end='')
            print('\r\n\r\n')
        #print("graphic")
            
    def start_play(self, player1, player2, start_player=0, is_shown=1):
        """
        start a game between two players
        """
        if start_player not in (0,1):
            raise Exception('start_player should be 0 (player1 first) or 1 (player2 first)')
        self.board.init_board(start_player)
        p1, p2 = self.board.players
        player1.set_player_ind(p1)
        player2.set_player_ind(p2)
        players = {p1: player1, p2:player2}
        if is_shown:
            self.graphic(self.board, player1.player, player2.player)
        #print("start_play")
        while(1):
            current_player = self.board.get_current_player()
            player_in_turn = players[current_player]
            move = player_in_turn.get_action(self.board)
            self.board.do_move(move)
            if is_shown:
                self.graphic(self.board, player1.player, player2.player)
            end, winner = self.board.game_end()
            if end:
                if is_shown:
                    if winner != -1:
                        print("Game end. Winner is", players[winner])
                    else:
                        print("Game end. Tie")
                return winner   
            
            
    def start_self_play(self, player, is_shown=0, temp=1e-3):
        """ start a self-play game using a MCTS player, reuse the search tree
        store the self-play data: (state, mcts_probs, z)
        """
        self.board.init_board()        
        p1, p2 = self.board.players
        states, mcts_probs, current_players = [], [], []
        #print("start_self_play")
        while(1):
            move, move_probs = player.get_action(self.board, temp=temp, return_prob=1)
            # store the data
            states.append(self.board.current_state())
            mcts_probs.append(move_probs)
            current_players.append(self.board.current_player)
            # perform a move
            self.board.do_move(move)
            if is_shown:
                self.graphic(self.board, p1, p2)
            end, winner = self.board.game_end()
            if end:
                # winner from the perspective of the current player of each state
                winners_z = np.zeros(len(current_players))  
                if winner != -1:
                    winners_z[np.array(current_players) == winner] = 1.0
                    winners_z[np.array(current_players) != winner] = -1.0
                #reset MCTS root node
                player.reset_player() 
                if is_shown:
                    if winner != -1:
                        print("Game end. Winner is player:", winner)
                    else:
                        print("Game end. Tie")
                return winner, zip(states, mcts_probs, winners_z)